In [9]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim

In [ ]:
english_stopwords = stopwords.words("english")
data = pd.read_csv(r"C:\Users\alkrd\Desktop\graduation_project\the_project\preprocssed_data\cleaned_data_USA.csv")
# data = pd.read_csv("../preprocessed_data/cleaned_data_Egypt.csv")    
# data = pd.read_csv("../preprocessed_data/cleaned_data_Turkey.csv")    


dataFrame = data[['newsSnippet']]

In [11]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    print("lemmatizing...")
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    text_out = []
    for text in texts:
        doc = nlp(text)
        new_text = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        final = " ".join(new_text)
        text_out.append(final)
    return text_out

lemmatized_texts = lemmatization(dataFrame['newsSnippet'].astype(str).tolist())
print("lemmitized sample: ", lemmatized_texts[100])

lemmatizing...
lemmitized sample:  yılbaşı özel şarkısını hatırlatarak şarkının biliyorsun diyerek


In [12]:
def gen_words(texts):
    print("removing stop words...")
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print("after removing stop words: ",data_words[100])

removing stop words...
after removing stop words:  ['yılbası', 'ozel', 'sarkısını', 'hatırlatarak', 'sarkının', 'biliyorsun', 'diyerek']


In [13]:
bigrams_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_tigram(texts):
    return (trigram[bigram[doc]] for doc in texts)

data_bigrams = list(make_bigrams(data_words))
data_bigrams_trigrams = list(make_tigram(data_bigrams))

print(data_bigrams_trigrams)



[['yeni', 'tv', 'kanalı', 'frekan', 'daha', 'haberimizde', 'maclarını', 'yayınlayacak', 'macları', 'canlı', 'evlerimize'], ['ile', 'ilgili', 'frekan', 'bilgilerine', 'ulasabilirsiniz', 'yeni', 'kanalı'], ['icildikten', 'sonra', 'ailesine', 'emri', 'peygamberin', 'kavliyle', 'oglumuz', 'dedi', 'kızın', 'annesi', 'yanıtını', 'verdi', 'aile', 'daha', 'sonra'], ['internet', 'fenomeni', 'programına', 've', 'talip', 'oldu', 'oyun', 'havası', 'oynayarak', 'giren', 'gulmekten', 've', 'sarkıları', 'esliginde', 'oynayan', 'programa', 'renk', 'kattı'], ['ev', 'sahibi', 'real', 'guclu', 'rakibine', 'bos', 'futbol', 'oynadı', 'ise', 'oyun', 'cıkamadı', 'yarı', 'sona_erdi'], ['bulunmasında', 'gok', 'bilginleri', 'cografyacılar', 'askerler', 'bircok', 'kimseler', 'tanırlar', 'gok', 'cezve', 'andırır', 'bu'], ['hakkında', 'estetiksiz', 'halde', 'yarısmasına', 'katıldıgına', 've', 'sonra'], ['yıldızı', 'artık', 'gollerine', 'bir', 'yenisi', 'ekledi', 'ozet', 'macının', 'zor', 'gunler', 'geciren'], ['be

In [14]:

id2word = corpora.Dictionary(data_bigrams_trigrams)
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.01
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]


In [15]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=5,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=20,
    alpha="auto"
)

In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.285998 -0.023993       1        1  31.360899
3      0.071241  0.243094       2        1  22.662772
1     -0.202642 -0.128542       3        1  15.901488
2     -0.189467  0.131046       4        1  15.316080
0      0.034870 -0.221605       5        1  14.758761, topic_info=              Term         Freq        Total Category  logprob  loglift
44              ve  1407.000000  1407.000000  Default  30.0000  30.0000
12             ile   352.000000   352.000000  Default  29.0000  29.0000
10            yeni   203.000000   203.000000  Default  28.0000  28.0000
167            son   203.000000   203.000000  Default  27.0000  27.0000
74             bir   143.000000   143.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
1181         lider    15.687466    16.326217   Topic5  -5.6152   1.8734
1895        yıldır    15.620299    16.263409   Topic5  -5.6195   1.8730
698   macını_canlı    15.188957    15.830021   Topic5  -5.6475   1.8720
3950        finale    14.398675    15.032046   Topic5  -5.7009   1.8703
1077      sifresiz    14.519936    15.165878   Topic5  -5.6926   1.8698

[181 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
994       4  0.993984  acıkladı
372       3  0.980359       ait
160       2  0.998757     aksam
720       4  0.975254      aldı
1379      3  0.968892    alındı
...     ...       ...       ...
288       2  0.997388       yıl
1895      5  0.983804    yıldır
82        3  0.989324   yıldızı
453       5  0.992178   yılında
201       1  0.987866     zaman

[151 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 3, 1])